In [1]:
from Bio import AlignIO

In [4]:
aln = AlignIO.parse("aln/dnaG.aln", 'fasta')
for a in aln:
    print(a)

SingleLetterAlphabet() alignment with 3 rows and 730 columns
----MYKLSEDEEKIINNIDIVDLIGQYVDLNKAGVSYKGYSPF...KGD Streptobacillus_moniliformis_61686.CDS_0
-----VIDQQTRQRILDSAQILDVVSDFISLRRQGVSYVGLCPF...IG- Porphyromonas_sp_57899.CDS_0
MTTGHFLPPEFLDELRQRITLSSLIGRTVKLTRAGREFRACCPF...EG- Zymomonas_mobilis_57566.CDS_0


In [5]:
a

<<class 'Bio.Align.MultipleSeqAlignment'> instance (3 records of length 730, SingleLetterAlphabet()) at 7f890c52c780>

In [6]:
print(a)

SingleLetterAlphabet() alignment with 3 rows and 730 columns
----MYKLSEDEEKIINNIDIVDLIGQYVDLNKAGVSYKGYSPF...KGD Streptobacillus_moniliformis_61686.CDS_0
-----VIDQQTRQRILDSAQILDVVSDFISLRRQGVSYVGLCPF...IG- Porphyromonas_sp_57899.CDS_0
MTTGHFLPPEFLDELRQRITLSSLIGRTVKLTRAGREFRACCPF...EG- Zymomonas_mobilis_57566.CDS_0


In [144]:
a = AlignIO.read("aln/dnaG.aln", 'fasta')
print(a)

SingleLetterAlphabet() alignment with 3 rows and 730 columns
----MYKLSEDEEKIINNIDIVDLIGQYVDLNKAGVSYKGYSPF...KGD Streptobacillus_moniliformis_61686.CDS_0
-----VIDQQTRQRILDSAQILDVVSDFISLRRQGVSYVGLCPF...IG- Porphyromonas_sp_57899.CDS_0
MTTGHFLPPEFLDELRQRITLSSLIGRTVKLTRAGREFRACCPF...EG- Zymomonas_mobilis_57566.CDS_0


In [145]:
print(a.format('clustal'))

CLUSTAL X (1.81) multiple sequence alignment


Streptobacillus_moniliformis_6      ----MYKLSEDEEKIINNIDIVDLIGQYVDLNKAGVSYKGYSPFKSENTP
Porphyromonas_sp_57899.CDS_0        -----VIDQQTRQRILDSAQILDVVSDFISLRRQGVSYVGLCPFHSDRNP
Zymomonas_mobilis_57566.CDS_0       MTTGHFLPPEFLDELRQRITLSSLIGRTVKLTRAGREFRACCPFHNEKTP

Streptobacillus_moniliformis_6      SFSVHPVKKIFKDFSSGKGGNVISFYSYIKNISYYEALHELSKKYGINIK
Porphyromonas_sp_57899.CDS_0        SFYVNPAKNICKCFSCGEGGTPLNFIMKHEQLSYSDALKYLAKKYNIEVI
Zymomonas_mobilis_57566.CDS_0       SFYVNDEKGFYHCFGCGAHGDAIRFVTETRGLSFMDAVRELAGQAGMDIP

Streptobacillus_moniliformis_6      KSNSKYLIKDNIG-----HKILKDAMEFFRLNFE--KSSEAKE----YLI
Porphyromonas_sp_57899.CDS_0        EREESEEEKQRNSERESLFIINDYAAKYFQTQLT--ETEEGRTVGLAYFR
Zymomonas_mobilis_57566.CDS_0       EANPAAVRQAKRT--ATLHDIMAEAAKWFSEQLHGIEGAEVRR----YLD

Streptobacillus_moniliformis_6      NRGFNLNDLKRYDIGYATNDWNLLYNHL-KEKYDVEELIKLGLVTVSSTD
Porphyromonas_sp_57899.CDS_0        ERGIRPETIERFGLGYAHESKTAFTEQAIKSGYPLKRLAEVGLS

In [149]:
import numpy as np
a_array = np.array([list(rec) for rec in a], np.character)
print("Array shape %i by %i" % a_array.shape)

Array shape 3 by 730


In [150]:
print(a_array)

[[b'-' b'-' b'-' ..., b'K' b'G' b'D']
 [b'-' b'-' b'-' ..., b'I' b'G' b'-']
 [b'M' b'T' b'T' ..., b'E' b'G' b'-']]


In [164]:
nseqs = len(a)
index = np.ones(a_array.shape[1], dtype=int)
for i in range(a_array.shape[1]):
    c = a_array[:,i]
    counts = np.unique(c, return_counts=True)
    
    # Remove unique columns
    if counts[0].shape == (1,):
        index[i] = 0
        continue
    
    # Count gaps
    ngaps = counts[1][b'-' == counts[0]]
    if ngaps / nseqs > 0.2:
        index[i] = 0
        continue
    
#     print(c)
index = np.array(index, dtype=bool)
filtered = a_array[:,index]

In [153]:
print(a_array.shape)
print(filtered.shape)

(3, 730)
(3, 493)


In [200]:
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

records = []
for i in range(filtered.shape[0]):
    seq = ''.join(np.array(filtered[i], dtype=str))
    name = a[i].id
    
    records.append(SeqRecord(Seq(seq, generic_dna), id=name))

new_aln = MultipleSeqAlignment(records)
print(new_aln)


DNAAlphabet() alignment with 3 rows and 493 columns
YKLSEDEEKIINNIDIVDLIGQYVDNKAVSYKGYSKSENTSHPV...VLK Streptobacillus_moniliformis_61686.CDS_0
VIDQQTRQRILDSAQILDVVSDFISRRQVSYVGLCHSDRNYNPA...TII Porphyromonas_sp_57899.CDS_0
FLPPEFLDELRQRITLSSLIGRTVKTRAREFRACCHNEKTYNDE...VDE Zymomonas_mobilis_57566.CDS_0


'Streptobacillus_moniliformis_61686.CDS_0'

In [184]:
cols_to_keep = np.argwhere(index)
new_aln = a[:,int(cols_to_keep[0])]
itercols = iter(cols_to_keep)
next(itercols)
for i in itercols:
    new_aln = new_aln + a[:,i]
    #print(a[:,int(i)])
    
    
    

ValueError: Invalid index

In [179]:
cols_to_keep = np.argwhere(index)
# print(cols_to_keep)
# print(a[:,0:5])
# print(a[:,0])
a[:,int(cols_to_keep[0])]

'YVF'

In [188]:
AlignIO.MultipleSeqAlignment?

In [139]:
a = a_array[:,0:5]
i = np.array([1,1,1,1,0], dtype=bool)
print(i)
print(a)
print(a[:,i])

[ True  True  True  True False]
[[b'-' b'-' b'-' b'-' b'M']
 [b'-' b'-' b'-' b'-' b'-']
 [b'M' b'T' b'T' b'G' b'H']]
[[b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-']
 [b'M' b'T' b'T' b'G']]


In [140]:
np.array(i, dtype=int)

array([1, 1, 1, 1, 0])

In [118]:
a[0] == b'-'

array([ True,  True,  True,  True, False], dtype=bool)

In [120]:
type(a[0] == b'-')

numpy.ndarray

In [51]:
counts[0][0]

b'-'

In [52]:
counts


(array([b'-', b'D'],
       dtype='|S1'), array([2, 1]))

In [58]:
counts[1][b'S' == counts[0]]

array([], dtype=int64)

True